In [7]:
file = '../txt/AML.txt'
outputfile = '../output_metric/Qualitative_metrics.csv'

In [ ]:
# 处理
import pandas as pd
# 读取CSV文件
csv_path = '../data.xlsx'
df = pd.read_excel(csv_path)

# 过滤出符合条件的定性指标
qualitative_metrics = df[df['type'] == '披露就得分']['metric'].tolist()

# # 将定性指标写入 metric_dx.txt
# with open('../output/metric_dx.txt', 'w') as file:
#     for metric in qualitative_metrics:
#         file.write(f"{metric}\n")


In [9]:
qualitative_metrics

2           Protected or restored habitats
4                       Absolute emissions
8                 Total energy consumption
10                Green financing projects
12               Green certified buildings
16                       water consumption
20                External audit conducted
26             Anti-corruption disclosures
27                Anti-corruption training
28      Assurance of sustainability report
29         List of relevant certifications
32    Availability of Healthcare Resources
33                Community Health Program
37               Philanthropic initiatives
39                  Controversial Sourcing
41              Consumer rights protection
50               Total number of employees
59            Community nutrition programs
Name: metric, dtype: object

In [12]:
import pandas as pd
from collections import defaultdict
# 加载字典 Excel 文件
dictionary_df = pd.read_excel("../dictionary_new1.xlsx")

# 初始化一个默认字典，每个指标对应多个关键词
metrics_keywords = defaultdict(list)

# 遍历每一行，将关键词加入对应的指标列表中
for _, row in dictionary_df.iterrows():
    metric = row["Metric"]
    keyword = row["keyword"]
    metrics_keywords[metric].append(keyword)

# 过滤字典，仅保留 qualitative_metrics 中的指标
# filtered_metrics_keywords = {metric: keywords for metric, keywords in metrics_keywords.items() if metric in qualitative_metrics}
filtered_metrics_keywords = {metric: metrics_keywords[metric] for metric in qualitative_metrics if metric in metrics_keywords}


In [14]:
import re
from transformers import pipeline
from collections import defaultdict
import pandas as pd

# 情感分析模型
sentiment_analyzer = pipeline('sentiment-analysis', model="distilbert-base-uncased-finetuned-sst-2-english")

# 你的metrics_keywords变量示例
filtered_metrics_keywords = defaultdict(list, filtered_metrics_keywords)

# 读取原文文本
with open(file, 'r') as file:
    raw_text = file.read()

# 存储匹配结果
results = []

# 遍历metrics_keywords中的每个指标及其关键词
for metric, keywords in filtered_metrics_keywords.items():
    found = False  # 标记是否找到匹配的关键词
    for keyword in keywords:
        if found:
            break  # 如果已经找到匹配项，则跳出循环
        keyword_pattern = re.escape(str(keyword))
        # match = re.search(rf"([^.]*?\b({keyword_pattern})\b[^.]*\.)", raw_text)
        match = re.search(rf"([^.]*?({keyword_pattern})[^.]*\.)", raw_text)

        if match:
            sentence = match.group(0)
            matched_keyword = match.group(2)  # 提取匹配的具体关键字
            
            # 将句子按句号分割，逐段分析
            sub_sentences = re.split(r'(?<=\.) ', sentence)
            for sub_sentence in sub_sentences:
                if len(sub_sentence) > 512:  # 如果子句仍然过长则截断
                    sub_sentence = sub_sentence[:512]

                # 进行情感分析
                sentiment = sentiment_analyzer(sub_sentence)
                confidence = sentiment[0]['score']
                sentiment_label = sentiment[0]['label']
                
                # 记录结果
                results.append({
                    'metric': metric,
                    'keyword': matched_keyword,
                    # 'sentence': sub_sentence,
                    # 'sentiment': sentiment_label,
                    'confidence': confidence
                })
                
            found = True  # 标记找到匹配项，退出关键词循环

# # 转换为DataFrame并保存结果
# results_df = pd.DataFrame(results)
# results_df.to_csv('qualitative_metrics_analysis_with_single_keyword.csv', index=False)
# print(results_df)


In [15]:
import pandas as pd

# 假设results已经存在，并包含每条记录的'Metric'字段
# 从results中提取Metric列，并将Value列设为1
metrics_data = [{'metric': result['metric'], 'value': 1, 'confidence':result['confidence']} for result in results]

# 创建新的DataFrame
metrics_df = pd.DataFrame(metrics_data).drop_duplicates()  # 去重，避免重复的Metric

# 保存或查看结果
metrics_df.to_csv(outputfile, index=False)
print(metrics_df)


                            metric  value  confidence
0               Absolute emissions      1    0.845853
1         Total energy consumption      1    0.992704
2  List of relevant certifications      1    0.983775
